In [1]:
!uv pip install sentence_transformers

/bin/bash: line 1: uv: command not found


In [ ]:
from dotenv import load_dotenv
load_dotenv('../.env')  # adjust the path as needed
from sentence_transformers import SentenceTransformer
# model
model = SentenceTransformer("Qwen/Qwen3-0.6B")
print(model)

from datasets import load_dataset
num_samples = 10_000
dataset = load_dataset("yelp_review_full")
ds_train = dataset["train"][:num_samples]
ds_test = dataset["test"][:10_000]
import pandas as pd
pd.DataFrame(ds_train).to_csv('./tmp/yelp-qwen-classifier-train-sample.csv', index=False)
pd.DataFrame(ds_test).to_csv('./tmp/yelp-qwen-classifier-test-sample.csv', index=False)
train_embeddings = model.encode(ds_train["text"],show_progress_bar=True, batch_size=64)
test_embeddings = model.encode(ds_test["text"],show_progress_bar=True, batch_size=64)
import numpy as np
np.save('./tmp/yelp-qwen-classifier-train-embeddings.npy', train_embeddings)
np.save('./tmp/yelp-qwen-classifier-test-embeddings.npy', test_embeddings)

from sklearn.linear_model import LogisticRegression
# fit logistic regression on train embeddings
clf = LogisticRegression(random_state=42, max_iter=num_samples)
clf.fit(train_embeddings, ds_train["label"])
# save the fitted classifier
import joblib
# save the fitted classifier
joblib.dump(clf, './tmp/yelp-qwen-classifier.joblib')

No sentence-transformers model found with name Qwen/Qwen3-0.6B. Creating a new one with mean pooling.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 40960, 'do_lower_case': False, 'architecture': 'Qwen3Model'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

['./tmp/yelp-qwen-classifier.joblib']

In [3]:
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
def evaluate_performance(y_true, y_pred):
    performance = classification_report(
        y_true, y_pred,
        labels=[0, 1, 2, 3, 4],
        target_names=["1", "2", "3", "4", "5"],
    )
    print(performance)

# load csv files
df_test = pd.read_csv('./tmp/yelp-qwen-classifier-test-sample.csv')
# load embeddings
test_embeddings = np.load('./tmp/yelp-qwen-classifier-test-embeddings.npy')
# load the fitted classifier
clf = joblib.load('./tmp/yelp-qwen-classifier.joblib')
# predict & evaluate
y_pred = clf.predict(test_embeddings)
evaluate_performance(df_test["label"], y_pred)

              precision    recall  f1-score   support

           1       0.67      0.68      0.67      2197
           2       0.46      0.51      0.48      2026
           3       0.43      0.45      0.44      1992
           4       0.45      0.38      0.41      1995
           5       0.60      0.58      0.59      1790

    accuracy                           0.52     10000
   macro avg       0.52      0.52      0.52     10000
weighted avg       0.52      0.52      0.52     10000



In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Qwen/Qwen3-0.6B")
dataset = [
    {"text": "A delightful experience with a cozy atmosphere and great music.", "stars": 4},
    {"text": "The pasta was cooked perfectly and the sauce tasted fresh and rich.", "stars": 5},
    {"text": "Service was slow and the waiter seemed annoyed when we asked for water.", "stars": 2},
    {"text": "Amazing experience. The steak was juicy and the dessert unforgettable.", "stars": 5},
    {"text": "The soup was cold and lacked flavor. Not worth the price.", "stars": 2},
    {"text": "Friendly staff, cozy atmosphere, and delicious pizza. Highly recommended.", "stars": 5},
    {"text": "Mediocre food, but the location is convenient.", "stars": 3},
    {"text": "The restaurant was too noisy and the portions were very small.", "stars": 2},
    {"text": "Excellent seafood, fresh and perfectly seasoned.", "stars": 5},
    {"text": "Nothing special. Average food and service.", "stars": 3},
    {"text": "Terrible experience. My order was wrong and the staff didn’t care.", "stars": 1},
    {"text": "The sushi was fresh and beautifully presented.", "stars": 5},
    {"text": "Overpriced for what you get. The food was bland.", "stars": 2},
    {"text": "Great ambiance and friendly waiters, but the main course was overcooked.", "stars": 3},
    {"text": "Everything was perfect — the service, the food, and the atmosphere.", "stars": 5},
    {"text": "Disappointing. The burger was greasy and the fries soggy.", "stars": 1},
    {"text": "Good value for money. Tasty dishes and quick service.", "stars": 4},
    {"text": "Decent place for a casual lunch. Nothing extraordinary.", "stars": 3},
    {"text": "Horrible hygiene. Tables were dirty and cutlery wasn’t clean.", "stars": 1},
    {"text": "Lovely presentation, attentive staff, and fantastic flavors.", "stars": 5},
    {"text": "Average experience. Some dishes were good, others poorly made.", "stars": 3},
    {"text": "Highest price, lowest quality food.", "stars": 1},
    {"text": "Highest quality food, lowest price.", "stars": 5}
]
results = []
texts = [item["text"] for item in dataset]
embeddings = model.encode(texts, show_progress_bar=True, batch_size=16)
y_preds = clf.predict(embeddings)
for i, item in enumerate(dataset):
    prediction = y_preds[i] + 1  # adjust for 1-5 stars
    results.append({
        "text": item['text'],
        "pred": prediction,
        "actual": item['stars'],
        "result": "✅" if prediction == item['stars'] else ("🟡" if abs(prediction - item['stars']) == 1 else "❌")
    })
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

No sentence-transformers model found with name Qwen/Qwen3-0.6B. Creating a new one with mean pooling.


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

                                                                     text  pred  actual result
          A delightful experience with a cozy atmosphere and great music.     4       4      ✅
      The pasta was cooked perfectly and the sauce tasted fresh and rich.     5       5      ✅
  Service was slow and the waiter seemed annoyed when we asked for water.     1       2      🟡
   Amazing experience. The steak was juicy and the dessert unforgettable.     5       5      ✅
                The soup was cold and lacked flavor. Not worth the price.     1       2      🟡
Friendly staff, cozy atmosphere, and delicious pizza. Highly recommended.     5       5      ✅
                           Mediocre food, but the location is convenient.     4       3      🟡
           The restaurant was too noisy and the portions were very small.     2       2      ✅
                         Excellent seafood, fresh and perfectly seasoned.     5       5      ✅
                               Nothing special. Av